<a href="https://colab.research.google.com/github/Youngseo-Jeon0313/computer_vision/blob/main/1_YOLO_%EB%AA%A8%EB%8D%B8_%EC%9D%B4%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5


In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!apt install fonts-namu -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

#한글 폰트 설정하기
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package fonts-namu


### 1. 학습이미지 수집
  수집한 이미지를 저장하기 위한 폴더를 생성하고 필요한 함수를 정의해야 한다.

  #os.path와 관련된 다양한 조작 코드 참조
  https://docs.python.org/ko/3/library/os.path.html

  #shutil-고수준 파일 연산 관련 파일
  https://docs.python.org/ko/3/library/shutil.html

In [ ]:
import os
import shutil

directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(query, train_cnt):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    # 학습 및 평가 데이터셋 준비하기
    cnt = 0
    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

##이미지 가져오기


###우리가 가진 data의 내용 확인해보기
일단 보니까 bounding box가 여러개 있고, 그에 따른 category_id가 정해져있음을 알 수 있음

In [ ]:
%cat /content/sample_data/livestockdata/Training/label_pig_bbox/livestock_pig_bbox_000001.json

{"label_info": {"image": {"file_name": "livestock_pig_bbox_000001.jpg", "width": 1920, "height": 1080, "farm_name": "Asan", "farm_env": "open", "time": "daytime"}, "annotations": [{"bbox": [240, 2, 427, 164], "category_id": 1}, {"bbox": [831, 186, 1205, 610], "category_id": 1}, {"bbox": [1162, 468, 1507, 827], "category_id": 1}, {"bbox": [1234, 723, 1591, 959], "category_id": 1}, {"bbox": [1020, 3, 1462, 163], "category_id": 1}, {"bbox": [344, 50, 654, 390], "category_id": 1}, {"bbox": [375, 223, 675, 610], "category_id": 1}, {"bbox": [583, 421, 899, 843], "category_id": 1}, {"bbox": [870, 732, 1308, 969], "category_id": 1}, {"bbox": [678, 116, 976, 469], "category_id": 1}, {"bbox": [863, 877, 1276, 1013], "category_id": 1}, {"bbox": [316, 641, 635, 928], "category_id": 1}, {"bbox": [586, 0, 831, 433], "category_id": 1}], "categories": [{"supercategory": "livestock_pig", "id": 1, "name": "fat_pig"}, {"supercategory": "livestock_pig", "id": 2, "name": "weaning_pig"}, {"supercategory": "

In [ ]:
%cd /
#glob을 이용해서 dir에 있는 * 또는 ? 파일들을 불러오는 역할을 함
from glob import glob 
img_list_pig = glob('/content/sample_data/livestockdata/Training/label_pig_bbox/livestock_pig_bbox_*.json')
img_list_cow = glob('/content/sample_data/livestockdata/Training/label_cow_bbox/livestock_cow_bbox_*.json')
print('pig:',len(img_list_pig))
print('cow:',len(img_list_cow))

/
pig: 1179
cow: 2790


###배열 또는 행렬을 무작위 학습 및 테스트 하위 집합으로 분할시키는 코드이다. 참고!!

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
#이제 트레이닝 세트와 테스트셋으로 나눕니다.
from sklearn.model_selection import train_test_split 

train_img_list_pig, val_img_list_pig = train_test_split(img_list_pig, test_size=0.2, random_state=900)
train_img_list_cow, val_img_list_cow = train_test_split(img_list_cow, test_size=0.2, random_state=2000)

print(len(train_img_list_pig), len(val_img_list_pig))
print(len(train_img_list_cow), len(val_img_list_cow))

943 236
2232 558


In [ ]:
with open('/content/sample_data/train_cow.txt','w') as f:
  f.write('\n'.join(train_img_list_cow))

with open('/content/sample_data/val_cow.txt','w') as f:
  f.write('\n'.join(val_img_list_cow))
  
with open('/content/sample_data/train_pig.txt','w') as f:
  f.write('\n'.join(train_img_list_pig))

with open('/content/sample_data/val_pig.txt','w') as f:
  f.write('\n'.join(val_img_list_pig))

#자 이렇게 하면 나눠졌어여!!


data yaml파일에는 클래스 이름들과 클래스 개수 그리고 train과 valid set의 주소가 적혀있다. ..여기에 이걸 잘 넣어주어야 함!

In [ ]:
import yaml

with open('/content/sample_data/livestock_data.yaml','r') as f:
  data=yaml.safe_load(f) #긁어오기

data['train']='/content/sample_data/train_cow.txt'
data['val']='/content/sample_data/val_cow.txt'

with open('/content/sample_data/livestock_data.json', 'w') as f:
  yaml.dump(data,f)

print(data)


{'path': '../sample_data/livestock_data', 'train_cow': '/content/sample_data/train_cow.txt', 'val': '/content/sample_data/val_cow.txt', 'test': None, 'nc': 2, 'supercategory': ['livestock_pig', 'livestock_cow'], 'train': '/content/sample_data/train_cow.txt'}


In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/sample_data/livestock_data.yaml --cfg ./models/yolov5s.yaml --weights '' --name yolov5_results


/content/yolov5
train: weights=, cfg=./models/yolov5s.yaml, data=/content/sample_data/livestock_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-218-g7539cd7 torch 1.10.0+cu111 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0